# Data Analysis
Analysis of line scans taken at the beam. See the `README` in the `Experimental Data` folder.

In [ ]:
import utils
from importlib import reload

from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import *

from scipy.interpolate import interp1d
from scipy.optimize import basinhopping

utils = reload(utils)
from utils import *

## Load Reference Data

In [ ]:
exp = parse_file('Reference Data/As_exp_standards_normalized.dat')
sim = parse_file('Reference Data/As_edge_sim_standards_phases_normalized.dat')

energy = np.array(sim['X1'], dtype=float)

sim_data_columns = [colm for colm in sim.columns if len(colm.replace('X', '')) > 3]
Sim_Refs = np.array(sim[sim_data_columns], dtype=float).T

exp_data_columns = [colm for colm in exp.columns if len(colm.replace('X', '')) > 3]
Exp_Refs = np.array(exp[exp_data_columns], dtype=float).T

refs = np.concatenate((Sim_Refs, Exp_Refs), axis=0)

Energy = np.linspace(11863, 11915, 5201)
Energy_exp = np.array([11866.5, 11870., 11873.5, 11876.5, 11879., 11885.5, 11900., 11904., 11909.5])

interperlator = interp1d(energy, refs)

Refs = interperlator(Energy)
Refs_exp = interperlator(Energy_exp)

print(f"Resolution: {Energy[1] - Energy[0]}")

data_columns = sim_data_columns + exp_data_columns

### Visualize References

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
for i, ref in enumerate(Refs):
    if i >= 20:
        c = i - 20
    else:
        c = i
    label = f'{data_columns[i]}'
    ax.plot(Energy, ref, '-', linewidth=3, c=plt.cm.tab20(c/19), label=label, alpha=1.)
ax.legend(fontsize=16, loc='center left', bbox_to_anchor=(1., .5), ncol=2, framealpha=1.)
plt.title("All references", fontsize=20)
format_axis(ax, ticks=(5,10))
plt.show()

## Analyze references taken at beam

In [ ]:
standard_fnames = ['lineAs_metal_scan006.mda.csv', 'lineCd3As2_scan008.mda.csv',
                   'lineAs2O3_scan009.mda.csv', 'lineAs3Te2_scan011.mda.csv']
standards = ['As metal', '$Cd_3As_2$', '$As_2O_3$', '$As_3Te_2$']
dfs = [pd.read_csv(f'Experimental data/Arsenic standards/{filename}') for filename in standard_fnames]
dfs = [preprocess_df(df) for df in dfs]

In [ ]:
def get_xy_from_colms(df, clm1, clm2):
    x = np.array(df[clm1], dtype=float)
    x = x * 1000  # convert to eV
    y = np.array(df[clm2], dtype=float)
    y = y - np.min(y)
    y = y / np.max(y)
    return x, y

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

for i, label in enumerate(standards):
    x, y = get_xy_from_colms(dfs[i], 'Energy (keV)', 'I_0 (a.u.)')
    ax.plot(x, y, '-', linewidth=3, c=plt.cm.tab20(i*2), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

x1, y1 = get_xy_from_colms(dfs[0], 'Energy (keV)', 'I_0 (a.u.)')
ax.plot(x1, y1, '-', linewidth=3, c=plt.cm.tab20(8), label=standards[0])

x2 = Energy
y2 = Refs[22]
y2 = y2 / np.max(y2)
label = 'As metal ASU stand.'

delta_E = x1[np.argmax(y1)] - x2[np.argmax(y2)]
print(delta_E)

ax.plot(x2, y2, '-', linewidth=3, c=plt.cm.tab20(12), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

x1, y1 = get_xy_from_colms(dfs[1], 'Energy (keV)', 'I_0 (a.u.)')
ax.plot(x1, y1, '-', linewidth=3, c=plt.cm.tab20(2), label=standards[1])

x2 = Energy
y2 = Refs[7]
y2 = y2 / np.max(y2)
label = '$Cd_3AS_2$ sim.'

delta_E = x1[np.argmax(y1)] - x2[np.argmax(y2)]
print(delta_E)

ax.plot(x2, y2, '-', linewidth=3, c=plt.cm.tab20(6), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

x1, y1 = get_xy_from_colms(dfs[2], 'Energy (keV)', 'I_0 (a.u.)')
ax.plot(x1, y1, '-', linewidth=3, c=plt.cm.tab20(4), label=standards[2])

x2 = Energy
y2 = Refs[24]
y2 = y2 / np.max(y2)
label = '$As_2O_3$ ASU stand.'

delta_E = x1[np.argmax(y1)] - x2[np.argmax(y2)]
print(delta_E)

ax.plot(x2, y2, '-', linewidth=3, c=plt.cm.tab20(0), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.show()

## Energy shift
A 2.73 eV energy shift will be applied to all simulated references so that is is aligned with the new experimental data.

In [ ]:
def spectral_loss(x, spectrum, target, metric):
    return eval(metric)(x*spectrum, target)

def get_scale(spectrum, target, metric='mean_absolute_error'):
    alpha0 = 0.1
    alpha = minimize(spectral_loss, alpha0, args=(spectrum, target, metric))['x']
    return alpha

In [ ]:
exp_energy = np.array(dfs[0]['Energy (keV)'], dtype=float)
print(min(exp_energy)*1000, max(exp_energy)*1000)
print(min(energy) + 2.74, max(energy) + 2.74)

In [ ]:
energy = energy + 2.74

bool_arr = exp_energy*1000 > 11865
Exp_Energy = exp_energy[bool_arr]*1000
Energy = Exp_Energy.copy()

interperlator = interp1d(energy, refs)

Refs = interperlator(Energy)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

x1, y1 = get_xy_from_colms(dfs[0], 'Energy (keV)', 'I_0 (a.u.)')
x1 = x1[bool_arr]
y1 = y1[bool_arr]
label = standards[0] + ' exp. stand.'
ax.plot(x1, y1, '-', linewidth=3, c=plt.cm.tab20(8), label=label)

x2 = Energy
y2 = Refs[22]
#y2 = y2 / np.max(y2)
label = 'As metal ASU ref.'

delta_E = x1[np.argmax(y1)] - x2[np.argmax(y2)]
print(delta_E)

alpha = get_scale(y2, y1)
print(alpha)
y2 = alpha*y2

ax.plot(x2, y2, '-', linewidth=3, c=plt.cm.tab20(12), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.savefig('Figures/As_metal_exp_comparison.png', dpi=800, transparent=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

x1, y1 = get_xy_from_colms(dfs[2], 'Energy (keV)', 'I_0 (a.u.)')
x1 = x1[bool_arr]
y1 = y1[bool_arr]
label = standards[2] + ' exp. stand.'
ax.plot(x1, y1, '-', linewidth=3, c=plt.cm.tab20(4), label=label)

x2 = Energy
y2 = Refs[24]
#y2 = y2 / np.max(y2)
label = '$As_2O_3$ ASU ref.'

delta_E = x1[np.argmax(y1)] - x2[np.argmax(y2)]
print(delta_E)

alpha = get_scale(y2, y1)
print(alpha)
y2 = alpha*y2

ax.plot(x2, y2, '-', linewidth=3, c=plt.cm.tab20(0), label=label)

ax.legend(fontsize=16, loc='upper right')
format_axis(ax, ticks=(5,10))
plt.savefig('Figures/As2O3_exp_comparison.png', dpi=800, transparent=True)

# Visualize the Experimental Data

In [ ]:
DATA = get_all_data()
# data is a list (by set) of a list (by pixel number) of dictaries (by xy pos)

nrows = 2
ncols = 3
fig, axes = plt.subplots(figsize=(16, 8), nrows=nrows, ncols=ncols)
plt.subplots_adjust(wspace=0.3, hspace=0.3)

for Set in range(1, 7):
    row = (Set - 1) // ncols
    colm = (Set - 1) % ncols
    ax = axes[row][colm]
    
    if Set <= 5:
        setdata = DATA[Set] 
        for i, pt in enumerate(setdata):
            intensity = np.array(pt['I'], dtype=float)
            ax.plot(pt['E'], intensity / 1000 + i*0.2, '-', linewidth=2, c=plt.cm.tab10(Set - 1))
    
        ax.annotate(f'Set {Set}', (0.7, 0.85), xycoords='axes fraction', fontsize=22)
        ax.set_ylabel("Intensity (arb. units)", fontsize=16)        
        ax.set_xlabel("Energy (eV)", fontsize=16)
        ax.tick_params(direction='in', width=2, length=8, which='major', axis='both')
        ax.set_xticklabels(np.array(ax.get_xticks(), dtype=int), fontsize=14)
        ax.set_yticklabels(np.array(ax.get_yticks(), dtype=float), fontsize=14)
        ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    else:
        ax.axis('off')

plt.savefig('Figures/linescans.png', dpi=800, transparent=True)

# Minimization of a reconstruction loss

# Supervised Machine Learning
Train a model to correlate dataset of linear combinations of the selected energies to the coefficients of the components.

In [ ]:
kwargs = {'N': 500, 'scale': 0.03, 'dropout': 0.9, 'training': False}
data, coeffs = generate_linear_combos(Refs, **kwargs)
kwargs = {'N': 50, 'scale': 0.03, 'dropout': 0.9, 'training': False}
test_data, test_coeffs = generate_linear_combos(Refs, **kwargs)

data = data * alpha
test_data = test_data * alpha

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
for i, ref in enumerate(Refs):
    if i >= 20:
        c = i - 20
    else:
        c = i
    label = f'{data_columns[i]}'
    ax.plot(Energy, ref, '-', linewidth=3, c=plt.cm.tab20(c/19), label=label, alpha=1.)
ax.legend(fontsize=16, loc='center left', bbox_to_anchor=(1., .5), ncol=2, framealpha=1.)
plt.title("All references", fontsize=20)
format_axis(ax, ticks=(5,10))
plt.show()